In [1]:
# from google.colab import drive

from utils.images import visualize

# drive.mount('/content/drive')
# data_path = '/content/drive/MyDrive/chest_xray/'

In [2]:
data_path = 'data/chest_xray'

Sto considerando di usare i seguenti modelli di ML:
1. ResNet.
2. GoogLeNet.
3. AlexNet.

In [3]:
import os
import pathlib

data_dir = pathlib.Path(os.path.join(data_path, 'train'))
total = len(list(data_dir.glob('**/*.jpeg')))
bacteria = len(list(data_dir.glob('**/BACTERIA-*.jpeg')))
virus = len(list(data_dir.glob('**/VIRUS-*.jpeg')))
P = len(list(data_dir.glob('**/NORMAL-*.jpeg')))
N = total - P
print(f'#training set = {total}')
print(f'#positives = {P}')
print(f'#negatives = {N} (bacteria = {bacteria}, virus = {virus})')

#training set = 5232
#positives = 1349
#negatives = 3883 (bacteria = 2538, virus = 1345)


Le immagini nel training set sono 5232 di cui: 1349 appartenenti alla classe NORMAL mentre 3883 alla classe PNEUMONIA.

```py
#training set = 5232
#positives = 1349
#negatives = 3883 (bacteria = 2538, virus = 1345)
```

Sto considerando diverse strategie per aumentare la dimensione del dataset:
1. Trasformazioni geometriche: specchiare, ruotare, tagliare, allungare.
   a. Specchiare e ruotare non mi sembrano che possano essere molto utili per questa tipologia di problema. Similmente anche
      il ritagliare l'immagine potrebbe rimuovere porzioni di immagini che potrebbero rivelarsi fondamentali per individuare o meno
      la polmonite.
   b. Tag
2. Modificare la luminosità.

È necessario ridimensionare le immagini per:
- Utilizzare meno risorse computazionali.
- Velocizzare sia addestramento che inferenza.
Al momento sto considerando le diverse strategie per il ridimensionamento delle immagini:
1. Ridimensionare tutte le immagini a 133x133 dove 133 è l'altezza minima tra tutte le immagini.
2. Ridimensionare tutte le immagini a 290x290 dove 290 è l'altezza media delle immagini

In [4]:
from utils.data import load_data
# (xs, ys) = load_data(data_path)
(xs133, ys133) = load_data(data_path, resize=(133, 133))
(xs190, ys190) = load_data(data_path, resize=(190, 190))

2025-02-02 17:27:40.637050: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-02 17:27:40.637472: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-02 17:27:40.639108: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-02 17:27:40.643402: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1738513660.650566  107183 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1738513660.65

In [5]:
import tensorflow as tf

xs133.shape

(5232, 133, 133, 1)

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_val, Y_train, Y_val = train_test_split(xs133, ys133, test_size=0.2, random_state=42)

In [7]:
from models.layers import RandomSwitch
from keras import layers

tr = layers.RandomTranslation(0.2, 0.2, data_format='channels_last')
rr = layers.RandomRotation(0.2, data_format='channels_last')
rb = layers.RandomBrightness(0.2, data_format='channels_last')
rc = layers.RandomContrast(0.2, data_format='channels_last')
# rz = layers.RandomZoom(0.2, 0.2, data_format='channels_last')
data_augmentation = RandomSwitch([tr, rr, rb, rc], 0.5)

2025-02-02 17:27:46.612936: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [8]:
from models import alex_net, resize

model = alex_net.create_model(133, 133, data_augmentation)
# resize130 = resize.Resize(133, 133)
# model_with_resize = alex_net.create_model(133, 133, data_augmentation=data_augmentation, resize=resize130)

/home/francesco/Documents/Università/ML/t5/.venv/lib/python3.12/site-packages/keras/src/layers/preprocessing/tf_data_layer.py:19: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [9]:
from keras import losses, metrics

loss = losses.categorical_crossentropy
metrics = metrics.binary_accuracy

model.compile(optimizer='adam', loss=loss, metrics=[metrics])
# model_with_resize.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [10]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ random_switch (RandomSwitch)    │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ rescaling (Rescaling)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ ?                      │   0 (unbuilt) │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ ?                      │   0 (unbuilt) │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ ?                      │   0 (unbuilt) │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ ?                      │   0 (unbuilt) │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ ?                      │   0 (unbuilt) │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [11]:
from keras import callbacks

# hist1 = model.fit(X_train, Y_train, epochs=10, validation_data=(X_val, Y_val), callbacks=[callbacks.EarlyStopping(patience=10)], verbose=1)
hist1 = model.fit(X_train, Y_train, batch_size=1, epochs=1, validation_data=(X_val, Y_val), verbose=1)


 176/4185 ━━━━━━━━━━━━━━━━━━━━ 4:40 70ms/step - binary_accuracy: 0.5848 - loss: 42.6813

KeyboardInterrupt: 